In [14]:
#일일매매내역 작성 및 전송
#rawadata 폴더에 FOS82105_YYYYMMDD 파일 수기로 저장 필요

import datetime
import os
import pandas as pd
import win32com.client as win32
import xlwings as xw

app = xw.App(visible=True, add_book=False)
today = datetime.date.today().strftime('%Y%m%d')
wb_path = r'K:\\부서 공유\\FI운용1부\\10. 개인별폴더\\김현수\\자동화\\일일매매내역_자동화.xlsb'
wb = app.books.open(wb_path)

# 매크로 실행
excel = wb.app
excel.api.Run('일일매매내역_생성')
excel.api.Run('일일매매내역_인쇄')
wb.close()

app.quit()


In [ ]:
# CD금리 PeerGroup

import xlwings as xw
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import urllib3
import ssl
# from requests.adapters import HTTPAdapter # SSLAdapter 클래스는 사용되지 않으므로 주석 처리 가능
# from urllib3.poolmanager import PoolManager # SSLAdapter 클래스는 사용되지 않으므로 주석 처리 가능
import time
import os # os 모듈 추가

# webdriver-manager의 SSL 검증 비활성화 (환경 변수 설정)
# 스크립트 상단 (import 다음)에 위치
os.environ['WDM_SSL_VERIFY'] = '0'

# Selenium 관련 라이브러리 추가
from selenium import webdriver
# Service 이름을 명확히 하기 위해 별칭(alias) 사용 (선택 사항이지만 권장)
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# webdriver-manager import 추가
from webdriver_manager.chrome import ChromeDriverManager


# InsecureRequestWarning 경고를 무시하도록 설정
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# SSLAdapter 클래스는 현재 코드에서 사용되지 않으므로 주석 처리 또는 삭제 가능
# class SSLAdapter(HTTPAdapter):
#     def __init__(self, ssl_context=None, **kwargs):
#         self.ssl_context = ssl_context
#         super().__init__(**kwargs)
#
#     def init_poolmanager(self, *args, **kwargs):
#         kwargs['ssl_context'] = self.ssl_context
#         return super().init_poolmanager(*args, **kwargs)

# create_headless_driver 함수는 현재 코드에서 호출되지 않으므로 주석 처리 또는 삭제 가능
# def create_headless_driver():
#     chrome_options = Options()
#     chrome_options.add_argument("--headless")
#     chrome_options.add_argument("--disable-gpu")
#     chrome_options.add_argument("--no-sandbox")
#     # chrome_driver_path = "/path/to/chromedriver" # 사용되지 않음
#     # service = Service(chrome_driver_path) # Service 대신 ChromeService 사용 권장
#     service = ChromeService(ChromeDriverManager().install()) # webdriver-manager 사용 예시
#     driver = webdriver.Chrome(service=service, options=chrome_options)
#     return driver

def get_shinhan_data():
    url_1 = "https://www.soletf.com/ko/fund/etf/211038?tabIndex=6"
    url_2 = "https://www.soletf.com/ko/fund/etf/211062?tabIndex=6"
    ytm_previous_1, duration_1, ytm_previous_2, duration_2 = "N/A", "N/A", "N/A", "N/A"
    try:
        response_1 = requests.get(url_1, verify=False, timeout=15) # timeout 추가
        response_1.raise_for_status()
        soup_1 = BeautifulSoup(response_1.text, 'html.parser')
        values_1 = soup_1.find_all('dd', class_='per')

        response_2 = requests.get(url_2, verify=False, timeout=15) # timeout 추가
        response_2.raise_for_status()
        soup_2 = BeautifulSoup(response_2.text, 'html.parser')
        values_2 = soup_2.find_all('dd', class_='per')

        if len(values_1) >= 3:
            ytm_previous_1 = values_1[1].text.strip()
            duration_1 = values_1[2].text.strip()
        if len(values_2) >= 3:
            ytm_previous_2 = values_2[1].text.strip()
            duration_2 = values_2[2].text.strip()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching Shinhan data: {e}")
    except Exception as e:
        print(f"Error processing Shinhan data: {e}")
    return ytm_previous_1, duration_1, ytm_previous_2, duration_2

def get_kb_data():
    url = "https://www.riseetf.co.kr/prod/finderDetail/44E4?searchFlag=viewtab1"
    try:
        response = requests.get(url, verify=False, timeout=15) # timeout 추가
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        h3_tag = soup.find('h3', class_='sub_page_title prod_title')
        if h3_tag:
            text = h3_tag.text
            ytm_match = re.search(r'ETF YTM:\s*([\d.]+)', text)
            duration_match = re.search(r'듀레이션:\s*([\d.]+)', text)
            if ytm_match and duration_match:
                etf_ytm = ytm_match.group(1) + '%'
                duration = duration_match.group(1)
                return etf_ytm, duration
    except requests.exceptions.RequestException as e:
         print(f"Error fetching KB data: {e}")
    except Exception as e:
         print(f"Error processing KB data: {e}")
    return "N/A", "N/A"

# get_hanwha_data() 함수 수정
def get_hanwha_data():
    url = "https://plusetf.co.kr/product/detail?n=006367"
    driver = None # driver 변수 초기화
    try:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        # SSL 오류 무시 옵션 추가
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        # User-Agent 추가 (선택 사항)
        options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36')

        # *** webdriver-manager 사용 ***
        service = ChromeService(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)

        driver.get(url)
        wait = WebDriverWait(driver, 15)
        # YTM 또는 Duration 관련 요소 중 하나라도 나타날 때까지 기다리기 (더 안정적)
        # 예를 들어, YTM 레이블이 포함된 컨테이너를 기다릴 수 있음 (선택자 조정 필요)
        wait.until(EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'YTM') or contains(text(), '듀레이션')]")))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # --- 데이터 추출 로직 (기존과 동일) ---
        ytm_value, duration_value = "N/A", "N/A"
        ytm_label = soup.find('span', string=re.compile("전일YTM"))
        if ytm_label:
            ytm_span = ytm_label.find_next('span') # find_next_sibling 이 더 안정적일 수 있음
            if ytm_span:
                 ytm_value_raw = ytm_span.text.strip()
                 try:
                     ytm_float = float(ytm_value_raw.strip('%'))
                     ytm_value = f"{ytm_float:.2f}%"
                 except ValueError:
                     ytm_value = "N/A" # 변환 실패 시 N/A
            else: ytm_value = "N/A" # 다음 span 못찾음
        else: ytm_value = "N/A" # 레이블 못찾음

        duration_label = soup.find('span', string=re.compile("듀레이션"))
        if duration_label:
            duration_span = duration_label.find_next('span')
            if duration_span:
                 duration_value = duration_span.text.strip()
            else: duration_value = "N/A"
        else: duration_value = "N/A"

        return ytm_value, duration_value

    except Exception as e:
        # WebDriverException, TimeoutException 등 구체적인 예외 처리 추가 가능
        print(f"Error fetching Hanwha data: {e}")
        return "N/A", "N/A"
    finally:
        # 드라이버 종료 구문 추가
        if driver:
            driver.quit()

def get_hana_data():
    url = "https://1qetf.com/pages/ETFproducts/ETF_info.view.php?etf_no=7"
    try:
        response = requests.get(url, verify=False, timeout=15) # timeout 추가
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        ytm, duration = "N/A", "N/A"
        ytm_label = soup.find('span', string=re.compile("YTM"))
        if ytm_label:
            ytm_element = ytm_label.find_next('span', class_='no-etfInfo__item-data')
            if ytm_element: ytm = ytm_element.text.strip()

        duration_label = soup.find('span', string=re.compile("듀레이션"))
        if duration_label:
            duration_element = duration_label.find_next('span', class_='no-etfInfo__item-data')
            if duration_element: duration = duration_element.text.strip()

        return ytm, duration
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Hana data: {e}")
    except Exception as e:
        print(f"Error processing Hana data: {e}")
    return "N/A", "N/A"

def get_kiwoom_data():
    url = "https://www.kiwoomam.com/fund/KI0202020000M?parentfundcode=3262"
    headers = { # User-Agent 최신화 권장
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers, verify=False, timeout=15) # timeout 추가
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        ytm, duration = "N/A", "N/A"
        duration_dt = soup.find('dt', string='Duration(년)')
        ytm_dt = soup.find('dt', string='YTM(%)')

        if duration_dt:
            duration_dd = duration_dt.find_next('dd')
            if duration_dd: duration = duration_dd.text.strip()
        if ytm_dt:
            ytm_dd = ytm_dt.find_next('dd')
            if ytm_dd: ytm = ytm_dd.text.strip()

        return ytm, duration
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Kiwoom data: {e}")
    except Exception as e:
        print(f"Error processing Kiwoom data: {e}")
    return "N/A", "N/A"

def get_samsung_data():
    # *** ChromeDriver 경로 지정 삭제 ***
    # chrome_driver_path = 'C:/chromedriver-win64/chromedriver.exe'
    driver = None # driver 변수 초기화
    try:
        # *** webdriver-manager 사용 ***
        service = ChromeService(ChromeDriverManager().install())
        options = Options() # Options 객체 생성 위치 변경
        # User-Agent 최신화 및 f-string 사용
        desktop_user_agent = ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                              'AppleWebKit/537.36 (KHTML, like Gecko) '
                              'Chrome/114.0.0.0 Safari/537.36')
        options.add_argument(f'user-agent={desktop_user_agent}')
        options.add_argument("window-size=1920,1080")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--headless')
        # SSL 오류 무시 옵션 추가
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')

        driver = webdriver.Chrome(service=service, options=options)

        url = 'https://www.samsungfund.com/etf/product/view.do?id=2ETFO1'
        driver.get(url)
        wait = WebDriverWait(driver, 30)
        # 페이지 로드 대기 조건 개선 (스크롤 전에 요소 기다리기)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.average-weight__item')))
        # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 스크롤이 필요 없을 수 있음
        # time.sleep(2) # 명시적 대기로 대체

        items = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.average-weight__item')))
        ytm = "N/A" # 기본값 N/A로 변경
        duration = "N/A" # 기본값 N/A로 변경

        for item in items:
            try:
                label_element = item.find_element(By.CSS_SELECTOR, 'span.average-weight__name')
                # label 값 가져오기 전에 요소 존재 확인 (더 안전)
                if label_element:
                    label = label_element.text.strip()
                    if label == '전일 YTM':
                        ytm_element = item.find_element(By.CSS_SELECTOR, 'div.average-weight__val strong')
                        if ytm_element: ytm = ytm_element.text.strip() + '%' # % 추가
                    elif label == '듀레이션(년)':
                        duration_element = item.find_element(By.CSS_SELECTOR, 'div.average-weight__val strong')
                        if duration_element: duration = duration_element.text.strip()
            except Exception as e: # NoSuchElementException 등 구체화 가능
                print(f"Samsung 항목 처리 중 오류 발생: {e}")

        # YTM/Duration 추출 실패 시 % 추가하지 않도록 수정
        # return (ytm + '%' if ytm != "N/A" else "N/A"), duration
        # -> ytm 값 자체에 % 추가하는 로직으로 변경했으므로 그대로 반환
        return ytm, duration

    except Exception as e:
        print(f"Samsung 데이터 가져오기 중 오류 발생: {e}")
        return "N/A", "N/A"
    finally:
        if driver:
            driver.quit()

def get_ace_data():
    # *** ChromeDriver 경로 지정 삭제 ***
    # chrome_driver_path = 'C:/chromedriver-win64/chromedriver.exe'
    driver = None # driver 변수 초기화
    try:
        # *** webdriver-manager 사용 ***
        service = ChromeService(ChromeDriverManager().install())
        options = Options() # Options 객체 생성 위치 변경
        desktop_user_agent = ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                              'AppleWebKit/537.36 (KHTML, like Gecko) '
                              'Chrome/114.0.0.0 Safari/537.36')
        options.add_argument(f'user-agent={desktop_user_agent}')
        options.add_argument("window-size=1920,1080")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--headless')
        # SSL 오류 무시 옵션 추가
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')

        driver = webdriver.Chrome(service=service, options=options)

        url = 'https://www.aceetf.co.kr/fund/K55101EC8482'
        driver.get(url)
        wait = WebDriverWait(driver, 30) # 대기 시간 유지 또는 조정

        # 페이지 로드 대기 조건 개선 (스크롤 전에 요소 기다리기)
        # wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        # time.sleep(2) # 명시적 대기로 대체

        # '듀레이션' 항목 찾기
        duration_label = wait.until(
           EC.presence_of_element_located((By.XPATH, "//dt[contains(text(), '듀레이션')]"))
        )
        duration_dd = duration_label.find_element(By.XPATH, "following-sibling::dd")
        duration = duration_dd.text.strip().replace('년', '').strip()

        # 'YTM (연환산)' 항목 찾기
        ytm_label = wait.until(
            EC.presence_of_element_located((By.XPATH, "//dt[text()='YTM (연환산)']"))
        )
        ytm_dd = ytm_label.find_element(By.XPATH, "following-sibling::dd")
        ytm = ytm_dd.text.strip() # YTM 값은 % 포맷팅 필요 시 추가

        return ytm, duration

    except Exception as e:
        print(f"ACE 데이터 가져오기 중 오류 발생: {e}")
        return "N/A", "N/A"
    finally:
        if driver:
            driver.quit()

def update_excel():
    # 데이터를 가져오는 함수 호출 (try-except 추가 권장)
    try: shinhan_ytm, shinhan_duration, shinhan_mm_ytm, shinhan_mm_duration = get_shinhan_data()
    except Exception as e: print(f"Shinhan Error: {e}"); shinhan_ytm, shinhan_duration, shinhan_mm_ytm, shinhan_mm_duration = "E", "E", "E", "E"
    try: kb_ytm, kb_duration = get_kb_data()
    except Exception as e: print(f"KB Error: {e}"); kb_ytm, kb_duration = "E", "E"
    try: hanwha_ytm, hanwha_duration = get_hanwha_data()
    except Exception as e: print(f"Hanwha Error: {e}"); hanwha_ytm, hanwha_duration = "E", "E"
    try: hana_ytm, hana_duration = get_hana_data()
    except Exception as e: print(f"Hana Error: {e}"); hana_ytm, hana_duration = "E", "E"
    try: kiwoom_ytm, kiwoom_duration = get_kiwoom_data()
    except Exception as e: print(f"Kiwoom Error: {e}"); kiwoom_ytm, kiwoom_duration = "E", "E"
    try: samsung_ytm, samsung_duration = get_samsung_data()
    except Exception as e: print(f"Samsung Error: {e}"); samsung_ytm, samsung_duration = "E", "E"
    try: hantu_ytm, hantu_duration = get_ace_data()
    except Exception as e: print(f"Hantu Error: {e}"); hantu_ytm, hantu_duration = "E", "E"

    # 어제 날짜 설정
    yesterday_dt = datetime.now() - timedelta(days=1)
    # 주말/휴일 처리 로직 (옵션)
    # if yesterday_dt.weekday() >= 5: # 토(5), 일(6)이면 금요일 데이터 처리 등
    #     print("Yesterday was weekend, adjust date or skip?")
    #     # yesterday_dt = ... # 날짜 조정 로직
    yesterday = yesterday_dt.strftime('%Y-%m-%d')

    # YTM 및 Duration 값 출력
    print("\n[YTM]")
    print(f"KB: {kb_ytm}")
    print(f"신한: {shinhan_ytm}")
    print(f"신한M: {shinhan_mm_ytm}")
    print(f"한화: {hanwha_ytm}")
    print(f"하나: {hana_ytm}")
    print(f"키움: {kiwoom_ytm}")
    print(f"삼성: {samsung_ytm}")
    print(f"한투: {hantu_ytm}")
    print("\n[Duration]")
    print(f"KB: {kb_duration}")
    print(f"신한: {shinhan_duration}")
    print(f"신한M: {shinhan_mm_duration}")
    print(f"한화: {hanwha_duration}")
    print(f"하나: {hana_duration}")
    print(f"키움: {kiwoom_duration}")
    print(f"삼성: {samsung_duration}")
    print(f"한투: {hantu_duration}")

    # 엑셀 업데이트 로직
    app = None
    wb = None
    try:
        # 엑셀 애플리케이션에 연결 (visible=False 권장)
        app = xw.App(visible=False, add_book=False)
        excel_path = r"K:\부서 공유\FI운용1부\머니마켓ETF\초단기ETF_PeerGroup.xlsx"
        wb = app.books.open(excel_path)
        ws = wb.sheets[0] # 첫 번째 시트 사용

        # A열에서 마지막 데이터 행 찾기 (4행부터 시작 가정)
        last_row_in_a = ws.range('A' + str(ws.cells.last_cell.row)).end('up').row
        next_row = max(4, last_row_in_a + 1) # 최소 4행부터 시작

        # 마지막 행 날짜 확인하여 덮어쓰기 또는 추가 결정 (이전 버전 로직 개선)
        if last_row_in_a >= 4:
             last_date_cell = ws.range(f'A{last_row_in_a}')
             try:
                 if isinstance(last_date_cell.value, datetime):
                     last_date_str = last_date_cell.value.strftime('%Y-%m-%d')
                 else:
                     # 문자열 형태의 날짜 파싱 시도
                     last_date_str = datetime.strptime(str(last_date_cell.value), '%Y-%m-%d').strftime('%Y-%m-%d')

                 if last_date_str == yesterday:
                     print(f"Data for {yesterday} already exists in row {last_row_in_a}. Overwriting.")
                     next_row = last_row_in_a # 덮어쓸 행 번호
                 else:
                     next_row = last_row_in_a + 1 # 다음 행에 추가
             except (ValueError, TypeError):
                 print(f"Could not parse date in cell A{last_row_in_a}. Appending to row {next_row}.")
                 # next_row는 이미 계산되었으므로 그대로 사용

        print(f"Target row for update: {next_row}")

        # 데이터 입력
        ws.range(f'A{next_row}').value = yesterday
        ws.range(f'B{next_row}').value = kb_ytm
        ws.range(f'C{next_row}').value = shinhan_ytm
        ws.range(f'D{next_row}').value = shinhan_mm_ytm
        ws.range(f'E{next_row}').value = hanwha_ytm
        ws.range(f'F{next_row}').value = hana_ytm
        ws.range(f'G{next_row}').value = kiwoom_ytm
        ws.range(f'H{next_row}').value = samsung_ytm
        ws.range(f'I{next_row}').value = hantu_ytm
        ws.range(f'J{next_row}').value = kb_duration
        ws.range(f'K{next_row}').value = shinhan_duration
        ws.range(f'L{next_row}').value = shinhan_mm_duration
        ws.range(f'M{next_row}').value = hanwha_duration
        ws.range(f'N{next_row}').value = hana_duration
        ws.range(f'O{next_row}').value = kiwoom_duration
        ws.range(f'P{next_row}').value = samsung_duration
        ws.range(f'Q{next_row}').value = hantu_duration

        # 숫자 서식 지정 (선택 사항) - 값 입력 후
        percent_cols = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        duration_cols = ['J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q']
        for col in percent_cols:
            cell = ws.range(f'{col}{next_row}')
            if isinstance(cell.value, str) and '%' in cell.value:
                try:
                    numeric_value = float(cell.value.strip('%')) / 100.0
                    cell.value = numeric_value
                    cell.number_format = '0.00%'
                except ValueError: pass # 변환 실패 시 그대로 둠
        for col in duration_cols:
            cell = ws.range(f'{col}{next_row}')
            try:
                 # 숫자로 변환 시도 후 서식 지정
                 numeric_value = float(cell.value)
                 cell.value = numeric_value
                 cell.number_format = '0.000' # 듀레이션 소수점 3자리로 가정
            except (ValueError, TypeError): pass # 변환 실패 시 그대로 둠


        wb.save()
        print(f"Successfully updated and saved Excel file: {excel_path}")

    except Exception as e:
        print(f"엑셀 업데이트 중 오류 발생: {e}")

    finally:
        # wb와 app 객체가 생성되었는지 확인 후 종료
        if wb:
            try: wb.close()
            except Exception as e: print(f"Workbook close error: {e}")
        if app:
            try: app.quit()
            except Exception as e: print(f"Excel app quit error: {e}")
        print("Excel operations finished.")

# 함수 호출 및 엑셀 업데이트
if __name__ == "__main__": # 스크립트 직접 실행 시에만 호출되도록 변경
    update_excel()


[YTM]
KB: 2.68%
신한: 2.77%
신한M: 2.42%
한화: 2.80%
하나: 2.70%
키움: 2.537%
삼성: 2.69%
한투: 2.65%

[Duration]
KB: 0.32
신한: 0.31
신한M: 0.07
한화: 0.27
하나: 0.21
키움: 0.403
삼성: 0.20
한투: 0.17
Target row for update: 250
Successfully updated and saved Excel file: K:\부서 공유\FI운용1부\머니마켓ETF\초단기ETF_PeerGroup.xlsx
Excel operations finished.


In [ ]:
#4JM40 입출금

import os
import win32com.client
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO

# 조회할 날짜를 지정 (없을시 오늘)
# specified_date = datetime.date(2024, 5, 28)
specified_date = datetime.date.today()

# Outlook MAPI에 연결
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item('localoutlook')

# '펀드오퍼레이션부' 폴더 참조 획득
fund_ops_folder = root_folder.Folders['펀드오퍼레이션부']

# 조건에 맞는 메일 검색
for message in fund_ops_folder.Items:
    if (message.ReceivedTime.date() == specified_date and
        '운용지시서(KDB생명)' in message.Subject):
        
        # 메일 내용 가져오기
        body = message.HTMLBody
        
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(body, 'html.parser')
        
        # 테이블 찾기
        tables = soup.find_all('table')
        
        # 첫 번째 테이블을 DataFrame으로 변환
        if tables:
            table = tables[0]
            df = pd.read_html(StringIO(str(table)))[0]
            
            # 수탁코드가 C04001인 행을 필터링
            filtered_df = df[(df.iloc[:, 2] == 'C04001') & 
                             (df.iloc[:, 4].isin(['입금소계', '출금소계']))]

            # 금액 및 D+1(예상금액) 열을 추출
            results = {
                "입금소계 금액": "0",
                "입금소계 D+1(예상금액)": "0",
                "출금소계 금액": "0",
                "출금소계 D+1(예상금액)": "0"
            }
            if not filtered_df.empty:
                for description in ['입금소계', '출금소계']:
                    row = filtered_df[filtered_df.iloc[:, 4] == description]
                    if not row.empty:
                        amount = row.iloc[0, 5]
                        d1_amount = row.iloc[0, 6]
                        results[f"{description} 금액"] = amount
                        results[f"{description} D+1(예상금액)"] = d1_amount
            
            # 출력하기
            print("4JM40 입출금")
            for key, value in results.items():
                # 숫자 값에 천 단위로 쉼표 추가
                formatted_value = f"{int(value):,}" if value.isdigit() else value
                print(f"{key}: {formatted_value}")
        else:
            print("4JM40 입출금")
            print("No table found in the email.")

4JM40 입출금
입금소계 금액: -
입금소계 D+1(예상금액): -
출금소계 금액: -
출금소계 D+1(예상금액): -


In [13]:
#예금보험기금 매매내역

#rawdata 폴더에 FOS82105 파일 필수
import os
import win32com.client
import datetime
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, Border, Side
import re

# 조회할 날짜 설정
specified_date = datetime.date.today()

# Outlook MAPI에 연결
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item('localoutlook')
trading_folder = root_folder.Folders['트레이딩']

# 결과를 저장할 리스트
all_results = []

# 아웃룩 메일에서 데이터 추출
for message in trading_folder.Items:
    if (message.ReceivedTime.date() == specified_date and
        ('4JM90' in message.Subject or '4JM91' in message.Subject)):
        
        body = message.HTMLBody
        soup = BeautifulSoup(body, 'html.parser')
        tables = soup.find_all('table')
        
        for table in tables:
            try:
                df = pd.read_html(StringIO(str(table)))[0]
                
                if df.shape[1] == 14:
                    df.columns = df.iloc[0]
                    df = df[1:]
                    
                    required_columns = ['운용펀드명', '종목명', '매매구분', '주문금액']
                    if all(col in df.columns for col in required_columns):
                        result_df = df[required_columns].copy()
                        result_df = result_df[result_df['운용펀드명'].notna()]
                        all_results.append(result_df)
            except:
                continue

# 펀드구분 추출 함수
def extract_fund_type(fund_name):
    if pd.isna(fund_name):
        return ""
    fund_name_str = str(fund_name)
    if '유니버스 11호' in fund_name_str:
        return '유니버스11호'
    elif '유니버스 12호' in fund_name_str:
        return '유니버스12호'
    else:
        pattern = r'유니버스\s*\d+호'
        match = re.search(pattern, fund_name_str)
        if match:
            return match.group().replace(' ', '')
        return ""

# 주문금액에서 숫자만 추출
def extract_amount(amount):
    if pd.isna(amount):
        return ""
    amount_str = str(amount).replace(',', '').replace('KRW', '').strip()
    try:
        amount_num = float(amount_str)
        if amount_num == int(amount_num):
            return int(amount_num)
        return amount_num
    except:
        return ""

# 아웃룩 메일 데이터 처리
outlook_data = []
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    
    for _, row in final_df.iterrows():
        outlook_data.append({
            '펀드구분': extract_fund_type(row['운용펀드명']),
            '거래구분': row['매매구분'] if pd.notna(row['매매구분']) else "",
            '종목명': row['종목명'] if pd.notna(row['종목명']) else "",
            '거래금리': "",
            '거래금액(액면기준)': extract_amount(row['주문금액'])
        })

# FOS82105 파일 읽기 (win32com 사용 - DRM 암호화 파일)
fos_path = r"K:\부서 공유\FI운용1부\10. 개인별폴더\김현수\rawdata"
fos_filename = f"FOS82105_{specified_date.strftime('%Y%m%d')}.xlsx"
fos_full_path = os.path.join(fos_path, fos_filename)

fos_data = []
if os.path.exists(fos_full_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False
    
    try:
        workbook = excel.Workbooks.Open(fos_full_path)
        worksheet = workbook.Worksheets(1)
        
        # 데이터가 있는 마지막 행 찾기
        last_row = worksheet.Cells(worksheet.Rows.Count, 5).End(-4162).Row  # E열 기준
        
        # 2행부터 마지막 행까지 읽기
        for row in range(2, last_row + 1):
            e_value = worksheet.Cells(row, 5).Value  # E열
            
            if e_value in ['4JM90', '4JM91']:
                fund_type = '유니버스11호' if e_value == '4JM90' else '유니버스12호'
                
                h_value = worksheet.Cells(row, 8).Value  # H열
                l_value = worksheet.Cells(row, 12).Value  # L열
                r_value = worksheet.Cells(row, 18).Value  # R열
                p_value = worksheet.Cells(row, 16).Value  # P열
                
                fos_data.append({
                    '펀드구분': fund_type,
                    '거래구분': h_value if h_value else "",
                    '종목명': l_value if l_value else "",
                    '거래금리': r_value if r_value else "",
                    '거래금액(액면기준)': p_value if p_value else ""
                })
        
        workbook.Close(False)
    finally:
        excel.Quit()

# 두 데이터 소스 병합
combined_data = outlook_data + fos_data

# 엑셀 파일 생성
wb = Workbook()
ws = wb.active

# 테두리 스타일 정의
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# 헤더 설정
headers = ['펀드구분', '거래구분', '종목명', '거래금리', '거래금액(액면기준)']
for col_num, header in enumerate(headers, 1):
    cell = ws.cell(row=1, column=col_num)
    cell.value = header
    cell.font = Font(name='맑은 고딕', size=11, bold=True)
    cell.alignment = Alignment(horizontal='center', vertical='center')
    cell.border = thin_border

# 데이터 입력
for row_num, data_row in enumerate(combined_data, 2):
    for col_num, (key, value) in enumerate(data_row.items(), 1):
        cell = ws.cell(row=row_num, column=col_num)
        cell.value = value
        cell.border = thin_border

# 열 너비 자동 조정
for column in ws.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass
    adjusted_width = min(max_length + 2, 50)
    ws.column_dimensions[column_letter].width = adjusted_width

# 파일 저장
save_path = r"K:\부서 공유\FI운용1부\일일매매내역"
if not os.path.exists(save_path):
    os.makedirs(save_path)

filename = f"예금보험기금_매매내역_{specified_date.strftime('%Y%m%d')}.xlsx"
full_path = os.path.join(save_path, filename)

wb.save(full_path)
wb.close()

In [ ]:
# 일임 설정해지내역 - KB손해보험
import win32com.client
from bs4 import BeautifulSoup
import openpyxl
import re
import datetime
import xlwings as xw  # xlwings 임포트 추가
import os
import time

def wait_for_file_access(file_path, timeout=10, check_interval=1):
    """
    파일이 접근 가능해질 때까지 대기합니다.
    timeout: 최대 대기 시간 (초)
    check_interval: 재시도 간격 (초)
    """
    start_time = time.time()
    while True:
        if os.path.exists(file_path):
            try:
                with open(file_path, 'rb'):
                    return True
            except IOError:
                pass
        if time.time() - start_time > timeout:
            return False
        time.sleep(check_interval)

# 오늘 날짜 사용
today = datetime.date.today()
today_str = today.strftime('%Y%m%d')
formatted_date = today.strftime('%Y-%m-%d')

today_excel_filename = f'KB손해보험설정해지_{today_str}.xlsx'
portfolio_file_path = f'K:/프로젝트/RPA/32.시가펀드 포트폴리오 자금판 작성/{formatted_date}/MOS_2215_{today_str}.xlsx'

# Outlook MAPI에 연결
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
root_folder = outlook.Folders.Item('LocalOutlook')

# '펀드오퍼레이션부' 폴더 참조 획득
fund_ops_folder = root_folder.Folders['펀드오퍼레이션부']

# 오늘 날짜의 이메일 검색
messages = fund_ops_folder.Items
messages.Sort("[ReceivedTime]", True)  # 최신 순으로 정렬

# 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active

# 헤더 작성
ws.append(['수익자', '구분', '처리일자', '인출일자', '판매사명', '펀드코드', '설정좌수', '설정금액', '해지좌수', '해지금액', '기준가(T-1)'])

# 포트폴리오 파일 열기
if not os.path.exists(portfolio_file_path):
    print(f"포트폴리오 파일이 존재하지 않습니다: {portfolio_file_path}")
    exit(1)

portfolio_wb = openpyxl.load_workbook(portfolio_file_path)
portfolio_ws = portfolio_wb.active

for message in messages:
    if today.strftime('%m/%d/%Y') in message.ReceivedTime.strftime('%m/%d/%Y') and 'KB손해보험' in message.Subject:
        # 수익자: 'KB손해보험'
        suikja = 'KB손해보험'
        # 구분 설정
        if '설정' in message.Subject:
            gubun = '설정'
        elif '해지' in message.Subject:
            gubun = '해지'
        else:
            continue  # '설정' 또는 '해지'가 없는 경우 스킵

        # HTML 본문 추출 및 텍스트 변환
        html_body = message.HTMLBody
        soup = BeautifulSoup(html_body, "html.parser")
        plain_text = soup.get_text(separator=" ", strip=True)

        # 데이터 추출을 위한 정규식 패턴 정의
        pattern = r'(\d{2})\s+(.*?)\s+([\d,]+)\s+([\d,]+)\s+(\d{4}-\d{2}-\d{2})(?:\s+(\d{4}-\d{2}-\d{2}))?\s+([\d,]+)\s+([\d,]+)'
        matches = re.finditer(pattern, plain_text)

        for match in matches:
            pmsamyung = match.group(2)
            # '무배당' 앞의 텍스트 제거
            index = pmsamyung.find('무배당')
            if index != -1:
                pmsamyung = pmsamyung[index:]
            else:
                continue

            # 판매사명에 '보험채권형' 또는 '배당주안정형' 또는 '인덱스플러스혼합형'이 포함되는지 확인
            if '보험채권형' in pmsamyung or '배당주안정형' in pmsamyung or '인덱스플러스혼합형' in pmsamyung:
                # 펀드코드 매핑
                if '보험채권형 1 호 ( 한투 )B 형' in pmsamyung:
                    fund_code = '0JM63'
                elif '보험채권형 1 호 ( 한투 )A 형' in pmsamyung:
                    fund_code = '0JM62'
                elif '배당주안정형 1 호 ( 한투 )A 형' in pmsamyung:
                    fund_code = '2JM04'
                elif '배당주안정형 1 호 ( 한투 )B 형' in pmsamyung:
                    fund_code = '2JM05'
                elif '인덱스플러스혼합형 1 호 ( 한투 )A 형' in pmsamyung:
                    fund_code = '0JM13'
                elif '인덱스플러스혼합형 1 호 ( 한투 )B 형' in pmsamyung:
                    fund_code = '0JM34'
                else:
                    fund_code = ''

                # 데이터 추가
                ws.append([
                    suikja,
                    gubun,
                    match.group(5),                 # 처리일자
                    match.group(6) if match.group(6) else '',  # 인출일자
                    pmsamyung,
                    fund_code,
                    match.group(3).replace(',', ''),  # 설정좌수
                    match.group(7).replace(',', ''),  # 설정금액
                    match.group(4).replace(',', ''),  # 해지좌수
                    match.group(8).replace(',', ''),  # 해지금액
                    ''  # 기준가(T-1) 초기값 (추후 xlwings로 채움)
                ])

# 데이터 형식 수정 및 천 단위 쉼표 추가
for row in ws.iter_rows(min_row=2, min_col=7, max_col=10):
    # J열에 계산식 추가 (I열 * K열)
    i_value = row[2].row  # I열의 행 번호
    k_value = row[3].row  # K열의 행 번호
    row[3].value = f'=I{i_value}*K{k_value}/1000'
    for cell in row[:4]:  # G, H, I, J 열의 값
        if isinstance(cell.value, str) and cell.value.replace(',', '').isdigit():
            cell.value = int(cell.value.replace(',', ''))
        # 숫자 형식 설정: 천 단위 쉼표, 0일 경우 '-'
        cell.number_format = '#,##0;-#,##0;-'

# openpyxl 작업 마무리
output_file_path = f'K:/부서 공유/FI운용1부/일임설정해지/KB손해보험설정해지_{today_str}.xlsx'
wb.save(output_file_path)
# 명시적으로 workbook을 삭제하여 메모리에서 해제
del wb, ws
print("설정해지내역 저장완료.")

# ---------------------- xlwings를 사용한 기준가(T-1) 매칭 추가 ----------------------

# xlwings 애플리케이션 초기화
app = None
kb_wb = None
portfolio_wb_xlwings = None
try:
    app = xw.App(visible=False)  # 백그라운드에서 실행
    app.display_alerts = False  # 알림 숨기기
    app.screen_updating = False  # 화면 업데이트 중지

    # KB손해보험설정해지 엑셀 열기
    kb_wb = app.books.open(output_file_path)
    kb_ws = kb_wb.sheets[0]  # 첫 번째 시트 선택

    # 포트폴리오 파일 열기
    if not os.path.exists(portfolio_file_path):
        print(f"포트폴리오 파일이 존재하지 않습니다: {portfolio_file_path}")
        # 필요 시 스크립트 종료 또는 예외 처리
        kb_wb.close()
        exit(1)

    portfolio_wb_xlwings = app.books.open(portfolio_file_path)
    portfolio_ws_xlwings = portfolio_wb_xlwings.sheets[0]  # 첫 번째 시트 선택

    # 포트폴리오 파일에서 펀드코드(B열)와 기준가(F열) 매핑 딕셔너리 생성
    fund_code_map = {}
    last_row_portfolio = portfolio_ws_xlwings.range('B' + str(portfolio_ws_xlwings.cells.last_cell.row)).end('up').row
    portfolio_data = portfolio_ws_xlwings.range(f'B2:F{last_row_portfolio}').value  # B2부터 F마지막 행까지
    for row_data in portfolio_data:
        if row_data[0] and row_data[4]:  # 펀드코드와 기준가가 모두 존재할 경우
            fund_code_map[str(row_data[0]).strip()] = row_data[4]

    # KB손해보험설정해지 엑셀에서 F열(펀드코드)와 K열(기준가) 업데이트
    last_row_kb = kb_ws.range('F' + str(kb_ws.cells.last_cell.row)).end('up').row
    for row_num in range(2, last_row_kb + 1):
        fund_code = str(kb_ws.range(f'F{row_num}').value).strip()
        기준가 = fund_code_map.get(fund_code, '')  # 매핑된 기준가 가져오기, 없으면 빈 문자열
        kb_ws.range(f'K{row_num}').value = 기준가

    # 엑셀 파일 저장 및 닫기
    kb_wb.save()
    print("기준가(T-1) 매칭 완료.")
finally:
    # 포트폴리오 파일 닫기
    if portfolio_wb_xlwings:
        portfolio_wb_xlwings.close()

    # KB손해보험설정해지 파일 닫기
    if kb_wb:
        kb_wb.close()

    # xlwings 애플리케이션 종료
    if app:
        app.quit()

In [15]:
# 시가펀드 자금판 오전 저장 (개선판)

import datetime
import xlwings as xw
import win32com.client
import os
import time

def get_file_paths():
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    origin_file = f"K:\\프로젝트\\RPA\\32.시가펀드 포트폴리오 자금판 작성\\{today}\\운용파일_{today.replace('-', '')}_RPA.xlsx"
    rawdata_file_1 = f"K:\\프로젝트\\RPA\\32.시가펀드 포트폴리오 자금판 작성\\{today}\\PMO\\PMO_통안채.xlsx"
    rawdata_file_2 = f"K:\\프로젝트\\RPA\\32.시가펀드 포트폴리오 자금판 작성\\{today}\\PMO\\PMO_국채.xlsx"
    save_file = f"K:\\부서 공유\\FI운용1부\\01. 1팀 자금판\\2025년\\운용파일_{today.replace('-', '')}_RPA.xlsb"
    save_directory = f"K:\\프로젝트\\RPA\\32.시가펀드 포트폴리오 자금판 작성\\REF\\참고\\{today}"
    return origin_file, rawdata_file_1, rawdata_file_2, save_file, save_directory

def add_vba_code(workbook):
    vba_code = """
    Sub 발행기관()
        Range("F3:F225").Copy
        Range("AE2").PasteSpecial Paste:=xlPasteValues
        Application.CutCopyMode = False
        ActiveSheet.Range("$AE$2:$AE$224").RemoveDuplicates Columns:=1
        ActiveWorkbook.Worksheets("펀드별조회").Sort.SortFields.Clear
        ActiveWorkbook.Worksheets("펀드별조회").Sort.SortFields.Add Key:=Range("AE2"), Order:=xlDescending
        With ActiveWorkbook.Worksheets("펀드별조회").Sort
            .SetRange Range("AE3:AE224")
            .Apply
        End With
        Range("AE2:AE224").Copy Range("U19")
    End Sub
    """
    
    vba_code2 = """
    Sub LinkMacroToButton()
        Dim btn As Button
        Set ws = ThisWorkbook.Sheets("펀드별조회")
        For Each btn In ws.Buttons
            If btn.Name = "Button 2" Then
                btn.OnAction = "발행기관"
            End If
        Next btn
    End Sub
    """
    
    VBAProject = workbook.VBProject
    VBAModule = VBAProject.VBComponents.Add(1)  # 1 = vbext_ct_StdModule
    VBAModule.CodeModule.AddFromString(vba_code)
    VBAModule.CodeModule.AddFromString(vba_code2)

def run_excel_operations():
    origin_file, rawdata_file_1, rawdata_file_2, save_file, save_directory = get_file_paths()
    
    # ExcelApp를 새로 생성하고 Visible을 True로 설정하여 기존 파일이 영향을 받지 않도록 처리
    ExcelApp = win32com.client.Dispatch("Excel.Application")
    ExcelApp.Visible = True  # Excel을 보이게 함
    
    # 덮어쓰기 시 경고창 무시
    ExcelApp.DisplayAlerts = False
    
    opened_files = []  # 새로 열린 파일들을 추적하기 위한 리스트
    
    # 파일 열기 (이미 열린 파일인지 확인 후, 열린 파일을 사용하도록 처리)
    try:
        origin_wb = ExcelApp.Workbooks.Open(origin_file)
        opened_files.append(origin_wb)
    except:
        origin_wb = None
        for wb in ExcelApp.Workbooks:
            if wb.FullName == origin_file:
                origin_wb = wb
                break
        if origin_wb is None:
            origin_wb = ExcelApp.Workbooks.Open(origin_file)
            opened_files.append(origin_wb)

    try:
        rawdata_wb_1 = ExcelApp.Workbooks.Open(rawdata_file_1)
        opened_files.append(rawdata_wb_1)
    except:
        rawdata_wb_1 = None
        for wb in ExcelApp.Workbooks:
            if wb.FullName == rawdata_file_1:
                rawdata_wb_1 = wb
                break
        if rawdata_wb_1 is None:
            rawdata_wb_1 = ExcelApp.Workbooks.Open(rawdata_file_1)
            opened_files.append(rawdata_wb_1)

    try:
        rawdata_wb_2 = ExcelApp.Workbooks.Open(rawdata_file_2)
        opened_files.append(rawdata_wb_2)
    except:
        rawdata_wb_2 = None
        for wb in ExcelApp.Workbooks:
            if wb.FullName == rawdata_file_2:
                rawdata_wb_2 = wb
                break
        if rawdata_wb_2 is None:
            rawdata_wb_2 = ExcelApp.Workbooks.Open(rawdata_file_2)
            opened_files.append(rawdata_wb_2)
    
    origin_ws = origin_wb.Worksheets("PMO(22200)")
    rawdata_ws_1 = rawdata_wb_1.Worksheets(1)
    rawdata_ws_2 = rawdata_wb_2.Worksheets(1)
    
    # 셀 복사 및 붙여넣기
    origin_ws.Range("Y8:AT52").Value = None
    origin_ws.Range("Y61:AT999").Value = None
    time.sleep(1)
    origin_ws.Range("A8:V52").Cut(origin_ws.Range("Y8"))
    origin_ws.Range("A61:V999").Cut(origin_ws.Range("Y61"))
    
    rawdata_ws_1.Range("A3:V50").Copy(origin_ws.Range("A8"))
    rawdata_ws_2.Range("A3:V999").Copy(origin_ws.Range("A61"))
    
    # 피벗 테이블 새로 고침
    origin_wb.Worksheets("펀드별조회").PivotTables("피벗 테이블2").PivotCache().Refresh()
    
    # VBA 코드 삽입 및 실행
    add_vba_code(origin_wb)
    origin_wb.Worksheets("펀드별조회").Activate()
    ExcelApp.Run("발행기관")
    ExcelApp.Run("LinkMacroToButton")
    
    # 파일 저장 (동일 이름 파일이 있어도 경고 없이 덮어쓰기)
    origin_wb.SaveAs(save_file, FileFormat=50)
    
    # 새 경로에 파일 저장
    os.makedirs(save_directory, exist_ok=True)
    app = xw.App(visible=False)
    app.display_alerts = False  # xlwings에서도 경고창 무시
    wb = app.books.open(save_file)
    wb.save(os.path.join(save_directory, '운용파일.xlsx'))
    wb.close()
    
    # 경고창 설정 원상복귀 후 엑셀 종료
    ExcelApp.DisplayAlerts = True
    
    # 새로 열린 파일들만 종료하기 위해 추적한 파일들만 닫음
    for wb in opened_files:
        wb.Close(SaveChanges=True)
    
    # ExcelApp은 종료하지 않고 남겨두기
    app.display_alerts = True
    app.quit()

if __name__ == "__main__":
    run_excel_operations()


In [ ]:
#시가펀드 자금판 오후 저장 (현재는 오전에 통합되어있음)

import os
import datetime
import xlwings as xw

#전일 누락시 다음날 아침에 이 부분 활성화 today 변수 수정
one_day = datetime.timedelta(days=1)
yesterday = datetime.datetime.today() - one_day
today = yesterday.strftime('%Y%m%d')
formatted_today = datetime.datetime.today().strftime('%Y-%m-%d')

# 파일 경로 정의
source_file_path = f'K:\\부서 공유\\FI운용1부\\01. 1팀 자금판\\2025년\\운용파일_{today}_rpa.xlsb'
dest_folder_path = f'K:\\프로젝트\\RPA\\32.시가펀드 포트폴리오 자금판 작성\\REF\\참고\\{formatted_today}'
dest_file_path = os.path.join(dest_folder_path, '운용파일.xlsx')

# 새로운 엑셀 인스턴스 생성
app = xw.App(visible=False)

# 엑셀 파일 열기
wb = app.books.open(source_file_path)
sheet = wb.sheets['Main_기존']

# 범위의 값을 삭제하고 셀 색상을 '색상 없음'으로 변경
for range_str in ['AG7:BC30', 'AG32:BC50', 'AG52:BC76']:
    range_obj = sheet.range(range_str)
    range_obj.value = None
    range_obj.color = None  # 셀의 배경색을 없앰

# 특정 범위의 값만 삭제
for range_str in ['AG31:BC31', 'AG51:BC51']:
    sheet.range(range_str).value = None

# 새 경로에 폴더 생성
os.makedirs(dest_folder_path, exist_ok=True)

# 경고 무시하고 파일 저장
wb.save(dest_file_path)

# 매크로 저장 안 함
wb.close()

# 엑셀 인스턴스를 닫는다
app.quit()



In [ ]:
#수시-본부 수탁고 업데이트

import xlwings as xw

# 사용자가 지정한 날짜 설정
# MOS2101 - 종합데이터 탭 - 펀드: FI운용본부 전체펀드 클릭 - 조회 후 F9 눌러 csv 파일로 source_csv 경로에 다운로드 후 specified_date에 날짜 입력하여 실행
 
specified_date = '20250824'  # 'YYYYMMDD' 형식으로 날짜를 입력.

# 파일 경로 설정
source_csv = f'K:\\부서 공유\\FI운용1부\\10. 개인별폴더\\김현수\\rawdata\\MOS2101_{specified_date}.csv'
target_xlsb = 'K:\\부서 공유\\FI운용1부\\10. 개인별폴더\\김현수\\자동화\\FI운용본부_수탁고양식.xlsb'

# xlwings 앱 인스턴스 생성, False로 백그라운드 작업가능
app = xw.App(visible=True)

try:
    # 소스 CSV 파일 열기
    wb_source = app.books.open(source_csv)
    sheet_source = wb_source.sheets[0]

    # 소스 데이터의 범위를 지정 (A열부터 IW열, 4행부터 데이터가 있는 마지막 행까지)
    last_row_source = sheet_source.range('A' + str(sheet_source.cells.last_cell.row)).end('up').row
    range_to_copy = sheet_source.range(f'A4:IW{last_row_source}').value

    # 타겟 Excel 파일 열기 (.xlsb 파일)
    wb_target = app.books.open(target_xlsb)
    sheet_target = wb_target.sheets['MOS2101']

    # A열에서 값이 있는 마지막 행 찾기
    last_row_target = sheet_target.range('A' + str(sheet_target.cells.last_cell.row)).end('up').row

    # 데이터를 타겟 시트의 적절한 위치에 붙여넣기
    sheet_target.range(f'A{last_row_target + 1}').value = range_to_copy

    # 변경 사항을 저장(하고 파일 닫기)
    wb_target.save()
    #wb_target.close()

finally:
    # 원본 파일 닫기 (변경 사항 없이)
    wb_source.close()

    # xlwings 앱 인스턴스 종료
    # app.quit()

print('데이터 복사 및 붙여넣기 완료.')


데이터 복사 및 붙여넣기 완료.
